In [1]:
using JuMP
using Gurobi
using DelimitedFiles
using CSV

## File handling

In [2]:
open("BP_parameters.txt") do file
    print(file)
end
a=readlines("BP_parameters.txt")
;

## Sets

In [3]:
I=2 # 1=gyn og 2=sarkom
U=2 # 1=hast (og 2=ikke-hast)
S=2 # 1 scanner
D=H=5 # 5 dager
R=5 # 5 radiologer
M=5 # 1=tolking intern, 2=tolking ekstern, 3=scanner, 4=MDT, 5=Åpen

;

## Parameters

In [4]:
#i rader, m kolonner.
T_im=[[1 1 1 1 1];
      [1 1 1 1 1]]

G_dr=[[2 2 2 2 2 2 2 2 2 2 2 2 2];
      [2 2 2 2 2 2 2 2 2 2 2 2 2];
      [2 2 2 2 2 2 2 2 2 2 2 2 2];
      [2 2 2 2 2 2 2 2 2 2 2 2 2];
      [2 2 2 2 2 2 2 2 2 2 2 2 2]]

N_sd=[[8 8 8 8 8];
      [8 8 8 8 8]]

S_sd=[[0 0 0 0 1];
      [0 0 0 0 1]]

D_iuW=[[5 5];
       [5 5]];

K_dr=[[8 8 8 8 8 8 8 8 8 8 8 8 8];
      [8 8 8 8 8 8 8 8 8 8 8 8 8];
      [8 8 8 8 8 8 8 8 8 8 8 8 8];
      [8 8 8 8 8 8 8 8 8 8 8 8 8];
      [8 8 8 8 8 8 8 8 8 8 8 8 8];]

D_iuD=[[0 0];
       [0 0]]

B_iu=[[0 0];
      [0 0]];

L_sd=[[1 1 1 1 1];
      [1 1 1 1 1]]

A_id=[[1 0 0 0 0];
      [0 0 1 0 0]]

P=3 # maks antall scannedager en radiolog kan være satt til

Q_d=1 # endring i daglig antall timer som er tillatt
Q_w=40

E_iu1=[[1 1];
       [1 1]]
E_iu2=[[1 1];
       [1 1]]
E_iu3=[[1 1];
       [1 1]]
E_iu4=[[1 1];
       [1 1]]
E_iu5=[[1 1];
       [1 1]]

E_iud=cat(dims=3,E_iu1,E_iu2,E_iu3, E_iu4, E_iu5)

;

## Variables

In [5]:
m = Model(with_optimizer(Gurobi.Optimizer))
@variable(m, delta_p[1:D,1:R] >= 0, Int)
@variable(m, delta_m[1:D,1:R] >= 0, Int)
@variable(m, x[1:I,1:U,1:S,1:D] >= 0, Int)
@variable(m, w[1:I,1:U,1:D,1:R,1:M] >= 0, Int)
@variable(m, u_i[1:I,1:U,1:D,1:H,1:R] >= 0, Int)
@variable(m, v[1:I,1:U,1:D,1:H,1:R] >= 0, Int)
@variable(m, alpha[1:I,1:U,1:D], Bin)
@variable(m, beta[1:S,1:D,1:R], Bin)
@variable(m, gamma[1:I,1:D,1:R], Bin)
@variable(m, zeta[1:S,1:D]>=0, Int)#number of open slots at the scanner s at day d
;

Academic license - for non-commercial use only


## Objective function

In [6]:
@objective(m, Min, sum(delta_p[d,r]+delta_m[d,r] for d=1:D, r=1:R ));

## Constraints

In [7]:
#Constraint2
@constraint(m,[d=1:D, r=1:R], delta_m[d,r]-delta_p[d,r] + sum(T_im[i,m]*w[i,u,d,r,m] for i=1:I,u=1:U, m=5)==G_dr[d,r])             
                 
#Constraint3
@constraint(m,[s=1:S,d=1:D], sum(x[i,u,s,d] for i=1:I,u=1:U)+zeta[s,d]==N_sd[s,d]-S_sd[s,d])
                            
#Constraint4
@constraint(m,[i=1:I,u=1:U], sum(x[i,u,s,d] for s=1:S,d=1:D)==D_iuW[i,u])

#Constraint5.1 NY
@constraint(m,[d=1:D,r=1:R], sum(T_im[i,m]*w[i,u,d,r,m] for i=1:I,u=1:U,m=1:M)>=K_dr[d,r]-Q_d)

#Constraint5.2 NY
@constraint(m,[d=1:D,r=1:R], sum(T_im[i,m]*w[i,u,d,r,m] for i=1:I,u=1:U,m=1:M)<=K_dr[d,r]+Q_d)

#Constraint5.3 NY - totalt antall timer en uke må være lik
@constraint(m,[r=1:R], sum(T_im[i,m]*w[i,u,d,r,m] for i=1:I,u=1:U, d=1:D, m=1:M)==Q_w)

#Constraint6
@constraint(m,[i=1:I,u=1:U], sum(alpha[i,u,d] for d=1:D)>=D_iuD[i,u])

#Constraint7
@constraint(m,[i=1:I,u=1:U, d=1:D], sum(x[i,u,s,d] for s=1:S)>=alpha[i,u,d])

#Constraint 8 NY
@constraint(m,[i=1:I,h=1:H, r=1:R], sum(u_i[i,u,d,h,r] for u=1:U,d=1:D) == sum(w[i,u,h,r,m] for u=1:U,m=1))

#Constraint 9 NY
@constraint(m,[i=1:I,h=1:H, r=1:R], sum(v[i,u,d,h,r] for u=1:U,d=1:D) == sum(w[i,u,h,r,m] for u=1:U, m=2))


#Constraint10 NY                                                                                             
for i=1:I, u=1:U, d=1:D
    if d+B_iu[i] <= 7
        @constraint(m, [i,u,d], sum(x[i,u,s,d] for s=1:S) - sum(u_i[i,u,d,h,r] for r=1:R,h=d:d+B_iu[i] if h <= D) == 0)
    else
        @constraint(m, [i,u,d], sum(x[i,u,s,d] for s=1:S) - sum(u_i[i,u,d,h,r] for r=1:R,h=d:d+B_iu[i] if h <= D) - sum(u_i[i,u,d,h,r] for r=1:R, h=1:(d+B_iu[i])%7 if h<d) == 0)                                                                                                                      
    end                                                                                                 
end
                                            
#Constraint11 NY                                                               
#@constraint(m,[i=1:I,u=1:U,d=1:D], sum(v[i,u,d,h,r] for r=1:R, h=(d:d+B_iu[i]) if h<=D)==E_iud[i,d])
#Nye constraint11                                                                                                                            
for i=1:I, u=1:U, d=1:D
    if d+B_iu[i,u] <= 7
        @constraint(m, [i,u,d], sum(v[i,u,d,h,r] for r=1:R, h=d:d+B_iu[i,u] if h<=D) == E_iud[i,u,d])
        @constraint(m, [i,u,d], sum(v[i,u,d,h,r] for r=1:R, h=1:d-1) == 0) #Setter starten av uka =0
        @constraint(m, [i,u,d], sum(v[i,u,d,h,r] for r=1:R, h=(d+B_iu[i,u]+1:D) if h <= D)==0)  #20                                                                                                          
    else                                                                                                                  
        @constraint(m, [i,u,d], sum(v[i,u,d,h,r] for r=1:R, h=d:d+B_iu[i,u] if h<=D) + sum(v[i,u,d,h,r] for r=1:R, h=1:(d+B_iu[i,u])%7 if h<d) == E_iud[i,u,d])
        @constraint(m, [i,u,d], sum(v[i,u,d,h,r] for r=1:R,h=((d+B_iu[i,u])%7) + 1:(d-1)) == 0)    #setter midten av uka =0                                                                                                                                    
    end                                                                                                 
end

#Constraint16 NY
@constraint(m, [i=1:I,d=1:D], sum(w[i,u,d,r,m] for u=1:U, r=1:R, m=4)==A_id[i,d])

#Constraint17
@constraint(m,[i=1:I,d=1:D],sum(gamma[i,d,r] for r=1:R)==A_id[i,d])
                
#Constraint18
@constraint(m,[i=1:I,d=1:D, r=1:R], sum(w[i,u,d,r,m] for u=1:U, m=4)==gamma[i,d,r])               

#Constraint19
@constraint(m, sum(w[i,u,d,r,m] for i=1:I, u=1:U, d=1:D, r=1:R, m=1)==sum(x[i,u,s,d] for i=1:I, u=1:U, s=1:S, d=1:D))                                                
                
            
#********scanning constraints*********                                                                                        
                                                                                                
#Constraint13 - Demand for radiologer på scannere på dag d
@constraint(m,[s=1:S,d=1:D], sum(beta[s,d,r] for r=1:R)==L_sd[s,d])

#En radiolog kan ikke være på to ulike scannere samme dag
@constraint(m, [r=1:R, d=1:D], sum(beta[s,d,r] for s=1:S)<=1)

                                                                                                
#Constraint15 - En radiolog kan ikke ha mer enn P scannetasks på en uke
@constraint(m,[r=1:R], sum(beta[s,d,r] for s=1:S, d=1:D)<=P) 
                                                                                      

#Constraint14: hvis en radiolog har scannevakt, kan hen ikke gjøre andre oppgaver den dagen. Hvis hen ikke har
#må summen av alle andre tasks bli K_dr
#@constraint(m,[d=1:D,r=1:R], 100*(1-sum(beta[s,d,r] for s=1:S)) >= sum(T_im[i,m]*w[i,u,d,r,m] for i=1:I, u=1:U, m =1:M if m!=3))

   
#Constraint12: For en gitt scanner en gitt dag for en gitt radiolog må 
#antallet scanneslots være like antall slots den dagen hvis radiologen er satt til scanning                                                                                      
@constraint(m,[d=1:D,r=1:R], sum(w[i,u,d,r,m] for i=1:I, u=1:U, m=3) == sum(beta[s,d,r]*N_sd[s,d] for s=1:S)) 
;                                                                                                                                                                                                        

## Results

In [8]:
@time begin
    optimize!(m)
    println("Run time: ")
end

println("Status: ",termination_status(m))
println("Primal status: ",primal_status(m))
println("Dual status: ",dual_status(m))
#println("Dual con1: ", shadow_price(con1[1]))
println("Objective value: ", objective_value(m))

println()
println("****** Variable: x ******")
for s=1:S
    println()
    println("Scanner: ",s)
    for d=1:D
        println()
        println("Day: ",d)
        println("__________________")
        for u=1:U
            for i=1:I
            if value(x[i,u,s,d]) != 0
                println("x[",i,u,s,d,"] = ", value(x[i,u,s,d]))
            end
            end
        end
        println("Open slots = ",value(zeta[s,d]))
    end
    
end
println()
println("****** Variable: w ******")
for d=1:D
    println()
    println("Day: ",d)
    println("__________________")
    for r=1:R
        #println("Patient group: ", i)
        for i=1:I
            for u=1:U
                #println("Radioligst: ",r)
                for m=1:M
                    if value(w[i,u,d,r,m]) != 0
                        println("w[",i,u,d,"(",r,")",m,"] = ", value(w[i,u,d,r,m]))
                    end
                end
            end
        end
    end
end
println()
println("****** Variable: u ******")
for i=1:I
    for u=1:U
        for d=1:D
            for h=1:H
                for r=1:R
                    if value(u_i[i,u,d,h,r]) != 0
                        println("u[",i,u,d,h,"(",r,")","] = ", value(u_i[i,u,d,h,r]))
                    end
                end
            end
        end
    end
end

println()
println("****** Variable: v ******")
for i=1:I
    for u=1:U
        for d=1:D
            for h=1:H
                for r=1:R
                    if value(v[i,u,d,h,r]) != 0
                        println("v[",i,u,d,h,"(",r,")","] = ", value(v[i,u,d,h,r]))
                    end
                end
            end
        end
    end
end

println()
println("****** Variable: delta ******")
for d =1:D
    println()
    println("Day: ",d)
    println("__________________")
    for r=1:R
        if value(delta_p[d,r]) != 0
            println("delta_p[",d,r,"] = ",value(delta_p[d,r]))
        end
        if value(delta_m[d,r]) != 0
            println("delta_m[",d,r,"] = ",value(delta_m[d,r]))
        end
    end
end

println()
println("****** Variable: alpha ******")
for i=1:I
    for u=1:U
        for d=1:D
            if value(alpha[i,u,d]) != 0
                println("alpha[",i,u,d,"] = ", value(alpha[i,u,d]))
            end
        end
    end
end

println()
println("****** Variable: beta ******")
for s=1:S
    for d=1:D
        for r=1:R
            if value(beta[s,d,r]) != 0
                println("beta[",s,d,"(",r,")","] = ", value(beta[s,d,r]))
            end
        end
    end
end

println()
println("****** Variable: gamma ******")
for i=1:I
    for d=1:D
        for r=1:R
            if value(gamma[i,d,r]) != 0
                println("gamma[",i,d,"(",r,")","] = ", value(gamma[i,d,r]))
            end
        end
    end
end

Academic license - for non-commercial use only
Optimize a model with 434 rows, 1720 columns and 4400 nonzeros
Variable types: 0 continuous, 1720 integer (120 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 4e+01]
Presolve removed 207 rows and 1295 columns
Presolve time: 0.00s
Presolved: 227 rows, 425 columns, 1385 nonzeros
Variable types: 0 continuous, 425 integer (185 binary)

Root relaxation: objective 2.800000e+01, 230 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   28.00000    0   49          -   28.00000      -     -    0s
H    0     0                      66.0000000   28.00000  57.6%     -    0s
H    0     0                      64.0000000   28.00000  56.2%     -    0s
     0     0   29.00000    0   48   64.00000   29.00000  54

## The model

In [9]:
#print(m)